![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.5.Clinical_Deidentification_in_French.ipynb)

# Clinical Deidentification in French

**Protected Health Information**:

Individual’s past, present, or future physical or mental health or condition
provision of health care to the individual
past, present, or future payment for the health care
Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import sys
import os
import json
import pandas as pd
import string
import numpy as np

import sparknlp
import sparknlp_jsl

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.util import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(SECRET, params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.4.2


# 1. French NER Deidentification Models
We have two different models you can use:
* `ner_deid_generic`, detects 7 entities
* `ner_deid_subentity`, detects 15 entities

### Creating pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d", "fr")\
    .setInputCols(["document","token"])\
	.setOutputCol("embeddings")

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


## 1.1. NER Deid Generic

**`ner_deid_generic`** extracts:
- Name
- Profession
- Age
- Date
- Contact (Telephone numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Organization)
- ID (Social Security numbers, Medical record numbers)

In [ ]:
ner_generic = MedicalNerModel.pretrained("ner_deid_generic", "fr", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_generic")

ner_converter_generic = NerConverter()\
    .setInputCols(["sentence","token","ner_deid_generic"])\
    .setOutputCol("ner_chunk_generic")

ner_deid_generic download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [ ]:
ner_generic.getClasses()

['O',
 'I-LOCATION',
 'I-CONTACT',
 'I-PROFESSION',
 'I-NAME',
 'I-DATE',
 'B-ID',
 'B-PROFESSION',
 'B-CONTACT',
 'I-ID',
 'B-NAME',
 'B-DATE',
 'B-LOCATION',
 'B-AGE',
 'I-AGE']

## 1.2. NER Deid Subentity

**`ner_deid_subentity`** extracts:

- Patient
- Doctor
- Hospital
- Date
- Organization
- City
- Street
- Username
- Profession
- Phone
- Country
- Age
- E-mail
- ZIP
- Medical Record

In [ ]:
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity", "fr", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_subentity")

ner_converter_subentity = NerConverter()\
    .setInputCols(["sentence", "token", "ner_deid_subentity"])\
    .setOutputCol("ner_chunk_subentity")

ner_deid_subentity download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [ ]:
ner_subentity.getClasses()

['O',
 'B-MEDICALRECORD',
 'B-ORGANIZATION',
 'I-PROFESSION',
 'B-DOCTOR',
 'B-USERNAME',
 'B-PROFESSION',
 'B-CITY',
 'B-DATE',
 'I-MEDICALRECORD',
 'B-E-MAIL',
 'B-PATIENT',
 'I-DOCTOR',
 'I-CITY',
 'I-DATE',
 'B-COUNTRY',
 'B-ZIP',
 'I-STREET',
 'I-PATIENT',
 'B-PHONE',
 'I-PHONE',
 'B-HOSPITAL',
 'B-STREET',
 'I-ORGANIZATION',
 'I-HOSPITAL',
 'B-AGE',
 'I-AGE',
 'I-COUNTRY']

## 1.3. Pipeline

In [ ]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_generic,
      ner_converter_generic,
      ner_subentity,
      ner_converter_subentity,
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
text = "J'ai vu en consultation Michel Martinez (49 ans), jardinier, adressé au Centre Hospitalier De Plaisir pour un diabète mal contrôlé avec des symptômes datant de Mars 2015."

text_df = spark.createDataFrame([[text]]).toDF("text")

result = model.transform(text_df)

### Results for `ner_generic`

In [ ]:
result.select(F.explode(F.arrays_zip('ner_chunk_generic.result', 'ner_chunk_generic.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------+----------+
|chunk                        |ner_label |
+-----------------------------+----------+
|Michel Martinez              |NAME      |
|49 ans                       |AGE       |
|jardinier                    |PROFESSION|
|Centre Hospitalier De Plaisir|LOCATION  |
|Mars 2015                    |DATE      |
+-----------------------------+----------+



### Results for `ner_subentity`

In [ ]:
result.select(F.explode(F.arrays_zip('ner_chunk_subentity.result', 'ner_chunk_subentity.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------+----------+
|chunk                        |ner_label |
+-----------------------------+----------+
|Michel Martinez              |PATIENT   |
|49 ans                       |AGE       |
|jardinier                    |PROFESSION|
|Centre Hospitalier De Plaisir|HOSPITAL  |
|Mars 2015                    |DATE      |
+-----------------------------+----------+



## DeIdentification

### Obfuscation mode

In [ ]:
# Downloading faker entity list.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/obfuscate_fr.txt

In [ ]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
      .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
      .setOutputCol("obfuscated") \
      .setMode("obfuscate")\
      .setObfuscateDate(True)\
      .setObfuscateRefFile('obfuscate_fr.txt')\
      .setObfuscateRefSource("file")

In [ ]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
deid_lp = LightPipeline(model)

In [ ]:
text = "J'ai vu en consultation Michel Martinez (49 ans), jardinier, adressé au Centre Hospitalier De Plaisir pour un diabète mal contrôlé avec des symptômes datant de Mars 2015."

In [ ]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

J'ai vu en consultation <PATIENT> (<AGE>), <PROFESSION>, adressé au <HOSPITAL> pour un diabète mal contrôlé avec des symptômes datant de <DATE>.


J'ai vu en consultation [*************] ([****]), [*******], adressé au [***************************] pour un diabète mal contrôlé avec des symptômes datant de [*******].


J'ai vu en consultation **** (****), ****, adressé au **** pour un diabète mal contrôlé avec des symptômes datant de ****.


J'ai vu en consultation Mme Leblanc (56), horticulteur, adressé au Centre Hospitalier De St Amand Les Eaux pour un diabète mal contrôlé avec des symptômes datant de 01-11-1975.


In [ ]:
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame(list(zip(result["masked_with_entity"], result["masked_with_chars"],
                          result["masked_fixed_length_chars"], result["obfuscated"])),
                 columns= ["Masked_with_entity", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked_with_entity,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"J'ai vu en consultation <PATIENT> (<AGE>), <PROFESSION>, adressé au <HOSPITAL> pour un diabète mal contrôlé avec des symptômes datant de <DATE>.","J'ai vu en consultation [*************] ([****]), [*******], adressé au [***************************] pour un diabète mal contrôlé avec des symptômes datant de [*******].","J'ai vu en consultation **** (****), ****, adressé au **** pour un diabète mal contrôlé avec des symptômes datant de ****.","J'ai vu en consultation Patriti Rhetat (87), Barrister's clerk, adressé au Centre Hospitalier Les Chanaux pour un diabète mal contrôlé avec des symptômes datant de 09-05-1987."


### Faker mode

In [ ]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setLanguage('fr')\
    .setObfuscateDate(True)\
    .setObfuscateRefSource('faker')

In [ ]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
deid_lp = LightPipeline(model)

In [ ]:
text = "J'ai vu en consultation Michel Martinez (49 ans), jardinier, adressé au Centre Hospitalier De Plaisir pour un diabète mal contrôlé avec des symptômes datant de Mars 2015."

In [ ]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

J'ai vu en consultation <PATIENT> (<AGE>), <PROFESSION>, adressé au <HOSPITAL> pour un diabète mal contrôlé avec des symptômes datant de <DATE>.


J'ai vu en consultation [*************] ([****]), [*******], adressé au [***************************] pour un diabète mal contrôlé avec des symptômes datant de [*******].


J'ai vu en consultation **** (****), ****, adressé au **** pour un diabète mal contrôlé avec des symptômes datant de ****.


J'ai vu en consultation Patriti Rhetat (87), Barrister's clerk, adressé au Centre Hospitalier Les Chanaux pour un diabète mal contrôlé avec des symptômes datant de 09-05-1987.


In [ ]:
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame(list(zip(result["masked_with_entity"], result["masked_with_chars"],
                          result["masked_fixed_length_chars"], result["obfuscated"])),
                 columns= ["Masked_with_entity", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked_with_entity,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"J'ai vu en consultation <PATIENT> (<AGE>), <PROFESSION>, adressé au <HOSPITAL> pour un diabète mal contrôlé avec des symptômes datant de <DATE>.","J'ai vu en consultation [*************] ([****]), [*******], adressé au [***************************] pour un diabète mal contrôlé avec des symptômes datant de [*******].","J'ai vu en consultation **** (****), ****, adressé au **** pour un diabète mal contrôlé avec des symptômes datant de ****.","J'ai vu en consultation Patriti Rhetat (87), Barrister's clerk, adressé au Centre Hospitalier Les Chanaux pour un diabète mal contrôlé avec des symptômes datant de 09-05-1987."


# 2. Pretrained French Deidentification Pipeline

- We developed a clinical deidentification pretrained pipeline that can be used to deidentify PHI information from French medical texts. The PHI information will be masked and obfuscated in the resulting text. 
- The pipeline can mask and obfuscate:
    - Patient
    - Doctor
    - Hospital
    - Date
    - Organization
    - Sex
    - City
    - Street
    - Country
    - ZIP
    - Username
    - Profession
    - Phone
    - Email
    - Age
    - ID number
    - Medical record number
    - Account number
    - SSN
    - Plate Number
    - IP address
    - URL

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification", "fr", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]


In [ ]:
text = """COMPTE-RENDU D'HOSPITALISATION
PRENOM : Jean
NOM : Dubois
NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058
ADRESSE : 18 Avenue Matabiau
VILLE : Grenoble
CODE POSTAL : 38000
DATE DE NAISSANCE : 03/03/1946
Âge : 70 ans 
Sexe : H
COURRIEL : jdubois@hotmail.fr
DATE D'ADMISSION : 12/12/2016
MÉDÉCIN : Dr Michel Renaud
RAPPORT CLINIQUE : 70 ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour.
Il nous a été adressé car il présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale.
L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV.
L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.
Hémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; les fonctions hépatiques et rénales étaient normales. PSA de 1,16 ng/ml.
ADDRESSÉ À : Dre Marie Breton - Centre Hospitalier de Bellevue Service D'Endocrinologie et de Nutrition - Rue Paulin Bussières, 38000 Grenoble
COURRIEL : mariebreton@chb.fr
"""

result = deid_pipeline.annotate(text)
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

COMPTE-RENDU D'HOSPITALISATION
PRENOM : [**]
NOM : [****]
NUMÉRO DE SÉCURITÉ SOCIALE : [***********]
ADRESSE : [****************]
VILLE : [******]
CODE POSTAL : [***]
DATE DE NAISSANCE : [********]
Âge : [****] 
Sexe : *
COURRIEL : [****************]
DATE D'ADMISSION : [********]
MÉDÉCIN : [**************]
RAPPORT CLINIQUE : **ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour.
** nous a été adressé car ** présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale.
L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome d

The results can also be inspected vertically by creating a Pandas dataframe as such:

In [ ]:
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame(list(zip(result["sentence"], result["masked"],
                          result["masked_with_chars"], result["masked_fixed_length_chars"], result["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,COMPTE-RENDU D'HOSPITALISATION,COMPTE-RENDU D'HOSPITALISATION,COMPTE-RENDU D'HOSPITALISATION,COMPTE-RENDU D'HOSPITALISATION,COMPTE-RENDU D'HOSPITALISATION
1,PRENOM : Jean,PRENOM : <PATIENT>,PRENOM : [**],PRENOM : ****,PRENOM : Mme Ollivier
2,NOM : Dubois,NOM : <PATIENT>,NOM : [****],NOM : ****,NOM : Mme Traore
3,NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058,NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>,NUMÉRO DE SÉCURITÉ SOCIALE : [***********],NUMÉRO DE SÉCURITÉ SOCIALE : ****,NUMÉRO DE SÉCURITÉ SOCIALE : 164033818514436
4,ADRESSE : 18 Avenue Matabiau,ADRESSE : <STREET>,ADRESSE : [****************],ADRESSE : ****,"ADRESSE : 731, boulevard de Legrand"
5,VILLE : Grenoble,VILLE : <CITY>,VILLE : [******],VILLE : ****,VILLE : Sainte Antoine
6,CODE POSTAL : 38000,CODE POSTAL : <ZIP>,CODE POSTAL : [***],CODE POSTAL : ****,CODE POSTAL : 37443
7,DATE DE NAISSANCE : 03/03/1946,DATE DE NAISSANCE : <DATE>,DATE DE NAISSANCE : [********],DATE DE NAISSANCE : ****,DATE DE NAISSANCE : 12/04/1946
8,Âge : 70 ans,Âge : <AGE>,Âge : [****],Âge : ****,Âge : 46
9,Sexe : H\nCOURRIEL : jdubois@hotmail.fr\nDATE D'ADMISSION : 12/12/2016,Sexe : <SEX>\nCOURRIEL : <E-MAIL>\nDATE D'ADMISSION : <DATE>,Sexe : *\nCOURRIEL : [****************]\nDATE D'ADMISSION : [********],Sexe : ****\nCOURRIEL : ****\nDATE D'ADMISSION : ****,Sexe : Femme\nCOURRIEL : georgeslemonnier@live.com\nDATE D'ADMISSION : 03/01/2017
